### import

In [1]:
import time, re, os, traceback, random
from os.path import join
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import date, datetime
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException, NoSuchWindowException # NoSuchElementException, UnexpectedAlertPresentException, WebDriverException,
# from datetime import timedelta
import functions

dtype = {'채무자키':str, '계좌키':str, '가상계좌번호':str, '법조치키':str}

In [2]:
driver, wait = None, None

class ErrorMsg(Exception):
    def __init__(self, message):
        super().__init__(message)
        self.message = message

    def __str__(self):
        return self.message

    
def 오류체크() :
    """오류(작업 대상이 아닌 예상치 못한 알람) 발생시 확인 버튼 눌러준 후 ErrorMsg(오류종류) 일으킴"""
    popups = driver.find_elements(By.CLASS_NAME, "w2popup_window") # 요소 없어도 에러 안나게, element가 아닌 -s. 없으면 빈리스트 반환
    if not popups : # 오류없는 경우
        return None
    
    # 메시지
    popup_message = popups[0].find_element(By.CLASS_NAME, "w2textbox").text
           
    # 확인버튼 누르기
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'confirm'))).click()
    mwt()

    return popup_message

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.2,0.5))

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def con_scourt() :
    driver = set_chrome_driver()
    # driver.get("https://ecfs.scourt.go.kr/psp/index.on?m=PSPA2GM01") # 제증명발급신청. 메인화면 하면 로그인 오류 생김
    driver.get("https://ecfs.scourt.go.kr/psp/index.on")
    
    driver.maximize_window() # 최대화 필수(상단 메뉴 보였다 안 보였다...)
    wait = WebDriverWait(driver, 30) 
    # 로그인 버튼
    # wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfheader_PSPMAMP01_wframe_btn_lgn'))).click() # 통합로그인 클릭
    wait.until(EC.element_to_be_clickable((By.ID, 'mf_pmf_content1_btnMainLogin'))).click() # 로그인 클릭
    mwt()
    return driver, wait

def certify_esc() :
    wait.until(EC.element_to_be_clickable((By.ID, "xwup_CancelButton"))).click() # 취소버튼 클릭
    
def certify(인증서:str, key:str) : 
    # 오류 나면 기다리지 말고 취소 후 다시 하도록 wait 일부러 안 씀
    lwt()
    elem = driver.find_element(By.ID, "xwup_certselect_tek_input1")
    swt()
    
    # 여러 인증서중 법원용 인증서 선택
    cert_elem_xpath = '//*[@id="xwup_cert_table"]/table/tbody/tr/td/div[text()="' + 인증서 + '"]'
    cert_elem = driver.find_element(By.XPATH, cert_elem_xpath)
    cert_elem.click()
    
    swt()
    elem.send_keys(key)
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def login(id:str, 인증서:str, key:str) : 
    
    # 아이디 입력란 찾기 및 입력
    input_element = driver.find_element("id", "mf_pfwork_ibx_elpUserIdForCert")
    input_element.send_keys(id)
    swt()
    
    # 로그인 버튼 클릭
    login_button = wait.until(EC.element_to_be_clickable((By.ID, 'mf_pfwork_btn_certlogin'))) # 로그인 클릭
    login_button.click()
    mwt()
    
    # 로그인(인증서)
    certify(인증서, key)
    lwt()
    
def select_court_event(court, event) :
    """법원 및 사건번호 입력"""
    
    # 법원 선택
    법원목록_id = "mf_pfwork_sbx_cortList" 
    사건구분_id = "mf_pfwork_acp_csDvs_input"
    사건일련번호_id = "mf_pfwork_ibx_csNo"
    
    Select(driver.find_element(By.ID, 법원목록_id)).select_by_visible_text(court)     
    swt() 
    
    
    
    # 사건번호 입력
    event = re.search(r"(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    
    Select(driver.find_element(By.ID, 'mf_pfwork_sbx_csYear')).select_by_visible_text(year) # 연도 선택
    element = driver.find_element(By.ID, 사건구분_id) # 사건구분 선택
    element.clear() 
    element.send_keys(sign)
    swt()

    element = driver.find_element(By.ID, 사건일련번호_id) # 사건일련번호 선택
    element.clear()
    element.send_keys(sn)
    swt()

### 나의 사건 조회

In [3]:
######################################
company = "솔림"    # 솔림, 대성
지점 = "광주"        # 광주, 서울, 부산
사건종류 = "확정"         # 확정,진행중,완료
보조pc = False
######################################
    
path_parquet_dir = join(r"\\192.168.0.75\자산관리팀\전산 dataset",company,r"기간축적데이터\나의사건\parquet")
path_parquet_file = functions.키워드로파일명찾기(path_parquet_dir, company+"나의사건전체")

if isinstance(path_parquet_file, list) :
    path_parquet_file = path_parquet_file[-1]
    
# 파일읽기 my_case
mc = pd.read_parquet(path_parquet_file)

In [4]:
display(mc.head(3))
print(mc.info())

,지점,법원,사건번호,재판부,사건지위,원고,피고,사건진행구분,확정일자,접수일자,기일시간,기일장소
0,광주,광주지방법원,2024가소619839,민사22단독,원고,주식회사 솔림헬프자산관리대부,안인섭,확정사건,2025-08-29,NaT,nan,nan
1,광주,광주지방법원,2025가소35735,민사22단독,원고,주식회사 솔림헬프자산관리대부,김철형,확정사건,2025-08-29,NaT,nan,nan
2,광주,광주지방법원,2025가소37597,민사22단독,원고,주식회사 솔림헬프자산관리대부,남일우,확정사건,2025-08-29,NaT,nan,nan


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19255 entries, 0 to 19254
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   지점      19255 non-null  object        
 1   법원      19255 non-null  object        
 2   사건번호    19255 non-null  object        
 3   재판부     19255 non-null  object        
 4   사건지위    19255 non-null  object        
 5   원고      19255 non-null  object        
 6   피고      19255 non-null  object        
 7   사건진행구분  19255 non-null  object        
 8   확정일자    8662 non-null   datetime64[ns]
 9   접수일자    10599 non-null  datetime64[ns]
 10  기일시간    19255 non-null  object        
 11  기일장소    19255 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 1.8+ MB
None


#### 로그인

In [ ]:
# 로그인 정보 불러오기
login_info_keys = ["전자소송_"+지점+"_"+x for x in ("id", "인증서", "pw")]
login_info_values = functions.get_info_from_decrypted_excel_dict(login_info_keys)

# 드라이버 실행
driver, wait = con_scourt()
login(login_info_values[0], login_info_values[1], login_info_values[2])

# 로그인 정보 메모리 삭제
login_info_values = [None] * len(login_info_values)
del login_info_values

### 엑셀 사건조회

#### 확정

In [ ]:
# 확정된사건 클릭
driver.find_element(By.ID, "mf_pfmenu_v3_li_a_150203").click()
swt()
# 사건번호 단추 클릭
driver.find_element(By.ID, "mf_pfwork_rad_choice_input_1").click()
swt()

확정건 = mc[mc.사건진행구분=="확정사건"]

#######################
temp = 확정건[:1]
temp["사건번호"] = "2025가단61983" # 에러용. 원래는 619839

for i, v in temp.iterrows():
#######################    

    select_court_event(v.법원, v.사건번호)
    
    # 조회버튼
    driver.find_element(By.ID, "mf_pfwork_btn_search").click()
    
    # 오류체크 및 조회log 작성 : 사건없는경우(아마도 법원이 잘못됐겠지)
    오류문구 = 오류체크()
    mc.loc[i, "사건조회결과"] = 오류문구 # 성공하면 None
    # 사건조회 후
    mc.loc[i, "사건조회일"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if 오류문구 :
        print(mc.loc[i, ["법원","사건번호","사건조회결과"]])
        continue
    
    

    # 일반내용 조회(사건번호 클릭)
    
        # 텍스트가 사건번호인 <a> 태그 찾기
    a_tag = driver.find_element(By.XPATH, "//a[text()='2024가소619839']").click()
    
    
    driver.find_element(By.id)
    
    
    
    

In [ ]:
select_court_event(v.법원, v.사건번호)
    
# 조회버튼
driver.find_element(By.ID, "mf_pfwork_btn_search").click()

In [ ]:
# 조회버튼
driver.find_element(By.ID, "mf_pfwork_btn_search").click()

In [ ]:
오류체크()

In [ ]:
message_element = driver.find_elements(By.CSS_SELECTOR, ".w2group.pop_contents .w2textbox") 
message_element
# popup_message = message_element.text
# popup_message

#### 진행

In [ ]:
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#### 완료
- cf) 관심사건은 확정 또는 진행에서 조회되나 완료사건은 완료에서만 조회됨
- 완료사건은 내가 진행또는 확정사건에서 지정하는 것임

### 홈-사건검색

In [ ]:
#######################
법원 = "서울중앙지방법원"
사건번호 = "2012가단249137"
당사자명 = "이오채"
#######################

# 사건구분입력모드 클릭
checkbox = driver.find_element(By.ID, "mf_pmf_content1_cbx_sagunNoGb_input_0")
if not checkbox.is_selected():
    checkbox.click()

# 법원
    # Select 객체 생성
select = Select(driver.find_element(By.ID, "mf_pmf_content1_sbx_cort5"))
    # 텍스트를 가진 옵션 선택
select.select_by_visible_text(법원)
swt()

# 사건번호, 당사자
사건번호_id = "mf_pmf_content1_ibx_csNoFull5"
당사자_input_id = "mf_pmf_content1_ibx_dangNm5"
for id, value in ((사건번호_id, 사건번호), (당사자_input_id, 당사자명)) : 
        # input 요소 찾기
    input_box = driver.find_element(By.ID, id)
        # 입력값 설정
    input_box.clear()  # 기존 값이 있다면 지우기
    input_box.send_keys(value)
    swt()
    
# 조회버튼 클릭
view_button = driver.find_element(By.ID, "mf_pmf_content1_btn_t5Search")
view_button.click()